# Introduction

This notebook demonstrates two simple simulation and analysis workflows. First, `hoomd_flowws` is used to perform a simulation of hard icosahedra using HOOMD's Monte Carlo support. We then visualize the system using `flowws-analysis` and `flowws-freud`.

## Simulation

We perform multiple steps of compressing the system and adjusting the Monte Carlo move size to make it easier to get the system to our final desired packing fraction.

In [ ]:
import flowws
import numpy as np

# this line can be used to clear the trajectory
# and backup files that have been generated
# !rm -f 3d_hpmc_icosahedra.*

shape = dict(type='icosahedron')

scope = dict(
    dump_filename='3d_hpmc_icosahedra.sqlite',
)

from hoomd_flowws import *

workflow = flowws.Workflow([
    ShapeDefinition(shape_arguments=[shape]),
    Init(number=256)] + [
        RunHPMC(steps=1e4, integrator='nvt', tune=(.2, 10, 100), compress_to=packing_fraction)
            for packing_fraction in np.linspace(.4, .54, 3)
    ] + [
    RunHPMC(steps=1e4, integrator='nvt', tune=(.2, 10, 100), compress_to=.6, dump_period=1e3),
], flowws.DirectoryStorage(), scope)

workflow.run();

# Visualization

We show three different ways to visualize the system. Select one depending on which mode best suits your use case.

## Directly within notebook

This mode is convenient for quick analysis, but depends on having vispy's notebook integration components [properly installed](http://vispy.org/installation.html).

In [ ]:
from flowws_analysis import *
from flowws_freud import *

flowws.Workflow([
    GTAR(filename=scope['dump_filename']),
    Steinhardt(num_neighbors=12),
    Colormap(),
    RDF(r_max=3),
    SmoothBOD(),
    Plato(outline=.02),
    ViewNotebook(),
], flowws.DirectoryStorage(), scope).run();

## Controls in notebook, plato visuals in popup windows

This mode is less reliant on details of your vispy installation. Jupyter is used to launch a local qt5 event loop, then visuals are made to pop up in their own windows while controls stay in the jupyter notebook. This depends on having the jupyter server running on your local machine in order to control windows.

In [ ]:
from flowws_analysis import *
from flowws_freud import *

%gui qt5

flowws.Workflow([
    GTAR(filename=scope['dump_filename']),
    Steinhardt(num_neighbors=12),
    Colormap(),
    RDF(r_max=3),
    SmoothBOD(),
    Plato(outline=.02),
    ViewNotebook(vispy_backend='pyside2'),
], flowws.DirectoryStorage(), scope).run();

## Standalone GUI window

This mode launches its own window showing all controls and analysis using the `ViewQt` module available in `flowws-analysis`. Of course, this command can also be run in the command line without a notebook.

In [ ]:
%%script sh

python -m flowws.run \
    GTAR --filename 3d_hpmc_icosahedra.sqlite \
    Steinhardt --num-neighbors 12 \
    Colormap \
    RDF --r-max 3 \
    SmoothBOD \
    Plato --outline 0.02 \
    ViewQt